## Programmierung des Algorithmus

Wir werden den Algorithmus k-means- nun kleinschrittig programmieren und jeden Schritt voneinander trennen. Dabei greifen wir auf keine externen Bibliotheken wie numpy zu. Lediglich für das Plotten der Ergebnisse benötigen wir matplotlib. Wir benötigen folgende Dinge